# Energy Price Cap

The [energy price cap](https://www.ofgem.gov.uk/information-consumers/energy-advice-households/check-if-energy-price-cap-affects-you) sets a cap on the standing and unit costs that energy suppliers can charge UK consumers for. It is calculated by Ofgem regularly (historically, every 6 months but every 3 since Q3 2022). This is different than the Energy Price Guarantee (EPG), which was introduced in Q3 2022 and subsidises energy bills for UK households by paying the difference between the energy price capped bill and a set lower amount.

PolicyEngine UK models the EPG as a subsidy that increases the incomes of UK households, using the following methodology:

1. Impute domestic energy consumption to the FRS from the LCFS.
2. Uprate domestic energy consumption using forecast price cap levels from Cornwall Insight.[^1]
3. For each month in a year:
    a. Divide total annual energy consumption by 12 to get monthly consumption.[^2]
    b. Find the price cap for that month.
    c. Find the price guarantee for that month.
    d. Calculate the relative difference between the price guarantee and the price cap: this is the subsidy rate.
    e. Multiply the subsidy rate by the monthly consumption to get the subsidy amount.
4. Sum total monthly subsidies to get the annual subsidy.

The chart below shows the parameter values for the price cap and price guarantee in PolicyEngine UK.

[^1]: [Cornwall Insight](https://www.cornwall-insight.com) is an energy research and consulting firm that produces public forecasts of Ofgem's energy price cap. They are not affiliated with Ofgem.

[^2]: Our model assumes energy consumption is evenly distributed throughout the year.

In [10]:
from policyengine_uk import CountryTaxBenefitSystem
import plotly.express as px
import pandas as pd

system = CountryTaxBenefitSystem()
ofgem = system.parameters.gov.ofgem

df = pd.DataFrame()

df["Date"] = [parameter.instant_str for parameter in ofgem.energy_price_cap.values_list]
df["Energy price cap"] = [parameter.value for parameter in ofgem.energy_price_cap.values_list]
df["Energy price guarantee"] = [parameter.value for parameter in ofgem.energy_price_guarantee.values_list]

px.line(
    df,
    x="Date",
    y=["Energy price cap", "Energy price guarantee"],
    title="Energy price cap and guarantee",
    color_discrete_sequence=["#BDBDBD", "#2C6496"],
).update_layout(
    yaxis_title="Average annual energy price",
    yaxis_tickprefix="£",
    yaxis_tickformat=",.0f",
    xaxis_title="Date",
    xaxis_tickformat="%Y-%m",
    height=600,
    width=800,
    legend_title_text="",
    template="plotly_white",
).update_traces(
    line_shape="hv",
)